In [27]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer

from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange

import scipy.io
from utils import log_gaussian_loss, gaussian, get_kl_Gaussian_divergence

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: {}'.format(device))


device: cuda


In [28]:
class BayesLinear_local_reparam(nn.Module):
    """Linear Layer where activations are sampled from a fully factorised normal which is given by aggregating
     the moments of each weight's normal distribution. The KL divergence is obtained in closed form. Only works
      with gaussian priors.
    """
    def __init__(self, n_in, n_out, prior):
        super(BayesLinear_local_reparam, self).__init__()
        self.n_in = n_in
        self.n_out = n_out
        self.prior = prior

        # Learnable parameters
        self.W_mu = nn.Parameter(torch.Tensor(self.n_in, self.n_out).uniform_(-0.05, 0.05))
        self.W_p = nn.Parameter(torch.Tensor(self.n_in, self.n_out).uniform_(-3, -2))
        self.b_mu = nn.Parameter(torch.Tensor(self.n_out).uniform_(-0.05, 0.05))
        self.b_p = nn.Parameter(torch.Tensor(self.n_out).uniform_(-3, -2))

    def forward(self, X, sample=False):
        #         print(self.training)

        if not self.training and not sample:  # This is just a placeholder function
            output = torch.mm(X, self.W_mu) + self.b_mu.expand(X.size()[0], self.n_out)
            return output, 0, 0

        else:

            # calculate std
            std_w = 1e-6 + F.softplus(self.W_p, beta=1, threshold=20)
            std_b = 1e-6 + F.softplus(self.b_p, beta=1, threshold=20)

            act_W_mu = torch.mm(X, self.W_mu)  # self.W_mu + std_w * eps_Ws
            act_W_std = torch.sqrt(torch.mm(X.pow(2), std_w.pow(2)))

            eps_W = self.W_mu.data.new(act_W_std.size()).normal_(mean=0, std=1)
            eps_b = self.b_mu.data.new(std_b.size()).normal_(mean=0, std=1)

            act_W_out = act_W_mu + act_W_std * eps_W  # (batch_size, n_output)
            act_b_out = self.b_mu + std_b * eps_b

            output = act_W_out + act_b_out

            # kld = KLD_cost(mu_p=0, sig_p=self.prior_sig, mu_q=self.W_mu, sig_q=std_w) + \
            #  KLD_cost(mu_p=0, sig_p=0.1, mu_q=self.b_mu, sig_q=std_b)

            KL_loss_weight = get_kl_Gaussian_divergence(self.prior.mu, self.prior.sigma**2, self.W_mu, std_w**2)
            KL_loss_bias = get_kl_Gaussian_divergence(self.prior.mu, self.prior.sigma**2, self.b_mu, std_b**2)
            KL_loss = KL_loss_weight + KL_loss_bias
            return output, KL_loss, 0

In [29]:
class bayes_linear_LR(nn.Module):
    def __init__(self, input_dim, output_dim, n_hid, prior_sig):
        super(bayes_linear_LR, self).__init__()

        self.prior_sig = prior_sig
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.bfc1 = BayesLinear_local_reparam(input_dim, n_hid, self.prior_sig)
        self.bfc2 = BayesLinear_local_reparam(n_hid, n_hid, self.prior_sig)
        self.bfc3 = BayesLinear_local_reparam(n_hid, n_hid, self.prior_sig)
        self.bfc4 = BayesLinear_local_reparam(n_hid, n_hid, self.prior_sig)
        self.bfc5 = BayesLinear_local_reparam(n_hid, output_dim, self.prior_sig)

      
        self.act = nn.Tanh()
        

    def forward(self, x, sample=False):
        tlqw = 0
        tlpw = 0

        # -----------------
        x, lqw, lpw = self.bfc1(x, sample)
        tlqw += lqw
        tlpw += lpw
        # -----------------
        x = self.act(x)
        # -----------------
        x, lqw, lpw = self.bfc2(x, sample)
        tlqw += lqw
        tlpw += lpw
        # -----------------
        x = self.act(x)
        # -----------------
        y, lqw, lpw = self.bfc3(x, sample)
        tlqw += lqw
        tlpw += lpw
        # -----------------
        x = self.act(x)
        # -----------------
        y, lqw, lpw = self.bfc4(x, sample)
        tlqw += lqw
        tlpw += lpw
        #------------------
        x = self.act(x)
        # -----------------
        y, lqw, lpw = self.bfc5(x, sample)
        tlqw += lqw
        tlpw += lpw

        return y, tlqw, tlpw

    def sample_predict(self, x, N_samples):
        # Just copies type from x, initializes new vector
        predictions = x.data.new(N_samples, x.shape[0], self.output_dim)
        tlqw_vec = np.zeros(N_samples)
        tlpw_vec = np.zeros(N_samples)

        for i in range(N_samples):
            y, tlqw, tlpw = self.forward(x, sample=True)
            predictions[i] = y
            tlqw_vec[i] = tlqw
            tlpw_vec[i] = tlpw

        return predictions, tlqw_vec, tlpw_vec
   

In [30]:
class BBP_Model_PINN_LR:
    def __init__(self, xt_lb, xt_ub, u_lb, u_ub,
                 input_dim, output_dim, no_units,
                 learn_rate, batch_size, no_batches,
                 prior_lambda1, prior_lambda2, num_epochs):

        
        self.xt_lb = torch.from_numpy(xt_lb).float().to(device)
        self.xt_ub = torch.from_numpy(xt_ub).float().to(device)
        self.u_lb = torch.from_numpy(u_lb).float().to(device)
        self.u_ub = torch.from_numpy(u_ub).float().to(device)


        self.learn_rate = learn_rate
        self.batch_size = batch_size
        self.no_batches = no_batches

        self.network = bayes_linear_LR(input_dim = input_dim, output_dim = output_dim,
                                        n_hid = no_units, prior_sig = gaussian(0, 1))

        # two PDE parameters
        self.log_noise_u = torch.log(torch.tensor([0.01], device = device))
        self.log_noise_f = torch.log(torch.tensor([0.01], device = device))
        # self.network.register_parameter('log_noise_u', self.log_noise_u)
        # self.network.register_parameter('log_noise_f', self.log_noise_f)

        self.prior_lambda1 = prior_lambda1
        self.prior_lambda2 = prior_lambda2

        self.lambda1_mus = nn.Parameter(torch.Tensor(1).uniform_(0, 2))
        self.lambda1_rhos = nn.Parameter(torch.Tensor(1).uniform_(-3, 2))
        self.lambda2_mus = nn.Parameter(torch.Tensor(1).uniform_(0, 0.05))
        self.lambda2_rhos = nn.Parameter(torch.Tensor(1).uniform_(-3, -2))

        self.network.register_parameter('lambda1_mu', self.lambda1_mus)
        self.network.register_parameter('lambda2_mu', self.lambda2_mus)
        self.network.register_parameter('lambda1_rho', self.lambda1_rhos)
        self.network.register_parameter('lambda2_rho', self.lambda2_rhos)

        self.network = self.network.to(device)

        # self.optimizer = torch.optim.SGD(self.network.parameters(), lr = self.learn_rate)
        self.optimizer = torch.optim.AdamW(self.network.parameters(), lr = self.learn_rate)
        # self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr = 1e-2, 
        #                                     steps_per_epoch = no_batches, epochs = num_epochs)
        self.loss_func = log_gaussian_loss

    def net_U(self, x, t):
        xt = torch.cat((x,t), dim=1)
        xt = 2*(xt-self.xt_lb)/(self.xt_ub-self.xt_lb) - 1
        u, KL_loss, _ = self.network(xt)
        return u, KL_loss

    def net_F(self, x, t, lambda1_sample, lambda2_sample):
        lambda_1 = lambda1_sample        
        lambda_2 = torch.exp(lambda2_sample)

        u, _ = self.net_U(x, t)
        u = u*(self.u_ub-self.u_lb) + self.u_lb # reverse scaling

        u_t = torch.autograd.grad(u, t, torch.ones_like(u),
                                    retain_graph=True,
                                    create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, torch.ones_like(u),
                                    retain_graph=True,
                                    create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x),
                                    retain_graph=True,
                                    create_graph=True)[0]

        # F = u_t + 1*u*u_x - 0.003*u_xx
        F = u_t + lambda_1*u*u_x - lambda_2*u_xx

        return F

    def fit(self, X, t, U, no_samples):
        self.network.train()

        U = (U-self.u_lb)/(self.u_ub-self.u_lb) # scaling

        # reset gradient and total loss
        self.optimizer.zero_grad()

        fit_loss_total = 0
        fit_loss_F_total = 0
        fit_loss_U_total = 0

        for i in range(no_samples):
            lambda1_epsilons = self.lambda1_mus.data.new(self.lambda1_mus.size()).normal_()
            lambda1_stds = torch.log(1 + torch.exp(self.lambda1_rhos))
            lambda2_epsilons = self.lambda2_mus.data.new(self.lambda2_mus.size()).normal_()
            lambda2_stds = torch.log(1 + torch.exp(self.lambda2_rhos))

            lambda1_sample = self.lambda1_mus + lambda1_epsilons * lambda1_stds
            lambda2_sample = self.lambda2_mus + lambda2_epsilons * lambda2_stds

            u_pred, KL_loss_para = self.net_U(X, t)
            f_pred = self.net_F(X, t, lambda1_sample, lambda2_sample)
            
            # calculate fit loss based on mean and standard deviation of output
            fit_loss_U_total += self.loss_func(u_pred, U, self.log_noise_u.exp(), self.network.output_dim)
            fit_loss_F_total += self.loss_func(f_pred, torch.zeros_like(f_pred), self.log_noise_f.exp(), self.network.output_dim)
            

        KL_loss_lambda1 = get_kl_Gaussian_divergence(self.prior_lambda1.mu, self.prior_lambda1.sigma**2, self.lambda1_mus, lambda1_stds**2)
        KL_loss_lambda2 = get_kl_Gaussian_divergence(self.prior_lambda2.mu, self.prior_lambda2.sigma**2, self.lambda2_mus, lambda2_stds**2)
        KL_loss_total = KL_loss_para + KL_loss_lambda1 + KL_loss_lambda2

        # minibatches and KL reweighting
        KL_loss_total = KL_loss_total/self.no_batches
        # expected loglikelihood
        total_loss = (KL_loss_total + fit_loss_U_total + fit_loss_F_total) / (no_samples*X.shape[0])

        
        total_loss.backward()
        self.optimizer.step()
        # self.scheduler.step()

        return fit_loss_U_total/no_samples, fit_loss_F_total/no_samples, KL_loss_total, total_loss

    def predict(self, xt, no_sample, best_net):
        xt = torch.tensor(xt, requires_grad=True).float().to(device)
        self.network.eval()
        sample = []
      
        for i in range(no_sample):
            
            xt = 2*(xt-self.xt_lb)/(self.xt_ub-self.xt_lb) - 1
            u_pred, _, _ = best_net(xt)
            u_pred = u_pred*(self.u_ub-self.u_lb) + self.u_lb # reverse scaling
            sample.append(u_pred.detach().cpu().numpy())
        return np.array(sample)


In [31]:
data = scipy.io.loadmat('./Data/burgers_shock.mat')

t = data['t'].flatten()[:,None] # 100 x 1
x = data['x'].flatten()[:,None] # 256 x 1
Exact = np.real(data['usol']).T # 100 x 256

X, T = np.meshgrid(x,t) # 100 x 256
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None])) # 25600 x 2
u_star = Exact.flatten()[:,None] # 25600 x 1

# Domain bounds of x, t
xt_lb = X_star.min(0)
xt_ub = X_star.max(0)

# training data
N_u = 2000
noise = 0.0
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx,:]
u_train = u_star[idx,:]

u_lb = u_train.min(0)
u_ub = u_train.max(0)

num_epochs, batch_size = 10000, len(X_u_train),

net = BBP_Model_PINN_LR(xt_lb, xt_ub, u_lb, u_ub,
                        input_dim = 2, output_dim = 1, no_units = 50, learn_rate = 1e-3,
                           batch_size = batch_size, no_batches = 1,
                           prior_lambda1=gaussian(0, 1), prior_lambda2=gaussian(0, 1), num_epochs = num_epochs)


In [32]:
fit_loss_U_train = np.zeros(num_epochs)
fit_loss_F_train = np.zeros(num_epochs)
KL_loss_train = np.zeros(num_epochs)
loss = np.zeros(num_epochs)

best_net, best_loss = None, float('inf')

X = torch.tensor(X_u_train[:,0:1], requires_grad = True, device = device).float()
t = torch.tensor(X_u_train[:,1:2], requires_grad = True, device = device).float()
U = torch.tensor(u_train, requires_grad = True, device = device).float()

X_u_test_25 = np.hstack([x, 0.25*np.ones_like((x))])
target_25 = Exact[25].reshape(-1,1)

In [33]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(comment = '_local_vi')

for i in range(num_epochs):

    EU, EF, KL_loss, total_loss = net.fit(X, t, U, no_samples = 20)
    
    fit_loss_U_train[i] = EU.item()
    fit_loss_F_train[i] = EF.item()
    KL_loss_train[i] = KL_loss.item()
    loss[i] = total_loss.item()

    writer.add_scalar("loss/total_loss", loss[i], i)
    writer.add_scalar("loss/U_loss", fit_loss_U_train[i], i)
    writer.add_scalar("loss/F_loss", fit_loss_F_train[i], i)
    writer.add_scalar("loss/KL_loss", KL_loss_train[i], i)
    

    if fit_loss_U_train[i] + fit_loss_F_train[i] < best_loss:
        best_loss = fit_loss_U_train[i] + fit_loss_F_train[i]
        best_net = copy.deepcopy(net.network)

    if i % 100 == 0 or i == num_epochs - 1:

        print("Epoch: {:5d}/{:5d}, total loss = {:.3f}, Fit loss U = {:.3f}, Fit loss F = {:.3f}, KL loss = {:.3f}".format(i + 1, num_epochs, 
               loss[i], fit_loss_U_train[i], fit_loss_F_train[i], KL_loss_train[i]))

    
        lambda1_mus = net.lambda1_mus.item()
        lambda1_stds = torch.log(1 + torch.exp(net.lambda1_rhos)).item()
        
        lambda2_mus = np.exp(net.lambda2_mus.item())
        lambda2_stds = torch.log(1 + torch.exp(net.lambda2_rhos)).item()

        noise_f = net.log_noise_u.exp().item()
        noise_u = net.log_noise_u.exp().item()
        
        samples_25 = net.predict(X_u_test_25, 100, net.network)
        u_pred_25 = samples_25.mean(axis=0)
        error_25 = np.linalg.norm(target_25-u_pred_25, 2)/np.linalg.norm(target_25, 2)

        samples_train = net.predict(X_u_train, 100, net.network)
        u_pred_train = samples_train.mean(axis=0)
        error_train = np.linalg.norm(u_train-u_pred_train, 2)/np.linalg.norm(u_train, 2)


        # writer.add_scalars("loss/train_test", {'train':error_train, 'test':error_25}, i)
        # writer.add_scalars("loss/f_u", {'noise_f':noise_f, 'noise_u':noise_u}, i)
       
        print("Epoch: {:5d}/{:5d}, lambda1_mu = {:.3f}, lambda2_mu = {:.3f}, lambda1_std = {:.3f}, lambda2_std = {:.3f}".format(i + 1, num_epochs,
                                                                                                                        lambda1_mus, lambda2_mus,
                                                                                                                        lambda1_stds, lambda2_stds))
        print("Epoch: {:5d}/{:5d}, error_25 = {:.5f}, error_train = {:.5f}".format(i+1, num_epochs, error_25, error_train))
        print("Epoch: {:5d}/{:5d}, noise_f = {:.5f}, noise_u = {:.5f}".format(i+1, num_epochs, noise_f, noise_u))
        print()

writer.close()

Epoch:     1/10000, total loss = 1343653.000, Fit loss U = 19905154.000, Fit loss F = 2667400192.000, KL loss = 17327.654
Epoch:     1/10000, lambda1_mu = 0.941, lambda2_mu = 1.030, lambda1_std = 0.530, lambda2_std = 0.124
Epoch:     1/10000, error_25 = 4.11953, error_train = 4.70155
Epoch:     1/10000, noise_f = 0.01000, noise_u = 0.01000

Epoch:   101/10000, total loss = 88789.672, Fit loss U = 2465759.250, Fit loss F = 175112752.000, KL loss = 16652.473
Epoch:   101/10000, lambda1_mu = 0.918, lambda2_mu = 0.974, lambda1_std = 0.520, lambda2_std = 0.120
Epoch:   101/10000, error_25 = 2.63963, error_train = 2.95371
Epoch:   101/10000, noise_f = 0.01000, noise_u = 0.01000

Epoch:   201/10000, total loss = 44031.957, Fit loss U = 1992445.875, Fit loss F = 86070672.000, KL loss = 16056.531
Epoch:   201/10000, lambda1_mu = 0.912, lambda2_mu = 0.953, lambda1_std = 0.517, lambda2_std = 0.119
Epoch:   201/10000, error_25 = 1.15260, error_train = 1.29181
Epoch:   201/10000, noise_f = 0.01000,

In [ ]:
X_u_test_25 = np.hstack([x, 0.25*np.ones_like((x))]); u_test_25 = Exact[25]
X_u_test_50 = np.hstack([x, 0.50*np.ones_like((x))]); u_test_50 = Exact[50]
X_u_test_75 = np.hstack([x, 0.75*np.ones_like((x))]); u_test_75 = Exact[75]



# aleatoric = best_net.log_noise.exp().cpu().data.numpy()
# epistemic = samples_25.var(axis = 0)**0.5
# total_unc = (aleatoric**2 + epistemic**2)**0.5

samples_25 = net.predict(X_u_test_25, 100, net.network)
samples_50 = net.predict(X_u_test_50, 100, net.network)
samples_75 = net.predict(X_u_test_75, 100, net.network)


u_pred_25 = samples_25.mean(axis = 0)
u_pred_50 = samples_50.mean(axis = 0)
u_pred_75 = samples_75.mean(axis = 0)


fig, axs = plt.subplots(1, 3, figsize = (15,4))
axs[0].plot(x,u_test_25, 'b-', linewidth = 2, label = 'Exact')
axs[0].plot(x,u_pred_25, 'r--', linewidth = 2, label = 'Prediction')
axs[0].set_xlabel('$x$')
axs[0].set_ylabel('$u(t,x)$')
axs[0].set_title('$t = 0.25$', fontsize = 10)
axs[0].axis('square')
#ax.set_xlim([-1.1,1.1])
#ax.set_ylim([-1.1,1.1])


axs[1].plot(x,u_test_50, 'b-', linewidth = 2, label = 'Exact')
axs[1].plot(x,u_pred_50, 'r--', linewidth = 2, label = 'Prediction')
axs[1].set_xlabel('$x$')
axs[1].set_ylabel('$u(t,x)$')
axs[1].set_title('$t = 0.5$', fontsize = 10)
axs[1].axis('square')


axs[2].plot(x,u_test_75, 'b-', linewidth = 2, label = 'Exact')
axs[2].plot(x,u_pred_75, 'r--', linewidth = 2, label = 'Prediction')
axs[2].set_xlabel('$x$')
axs[2].set_ylabel('$u(t,x)$')
axs[2].set_title('$t = 0.75$', fontsize = 10)
axs[2].axis('square')



